In [1]:
# Import general libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%%time
# Import velocity field
import scipy.io as sio

print("Import data")
#Import velocity data from file in data-folder
mat_file = sio.loadmat('./data/Files/Bickley_jet.mat')

U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time = mat_file['t']

Import data
Wall time: 41.9 ms


In [3]:
# Create meshgrid
X, Y = np.meshgrid(x, y)

In [4]:
# Name of Dataset. {"Ocean", "General"}
Type = "General"

# Number of cores to be used for parallel computing
Ncores = 4

# Specify whether the flow is incompressible. {True, False}
incompressible = True

# Specify periodic boundary conditions
periodic_boundary = [False, False]

# Summarize list of parameters of the flow.
params_data = Type, X, Y, time, U, V, Ncores, incompressible, periodic_boundary

In [5]:
%%time
from ipynb.fs.defs.System import *
DS = System()
DS._init_data(params_data)

Wall time: 72.8 ms


In [6]:
DS._Interpolation_velocity()

In [7]:
%%time

# Initial time (in days)
t0 = DS.Time[0, 0]

# Final time (in days)
tN = DS.Time[0, 20]

# time step-size
dt = 0.2

time = np.arange(t0, tN+dt, dt)

# longitudinal and latitudinal boundaries (in degrees)
xmin = np.min(DS.X)
xmax = np.max(DS.X)/5
ymin = np.min(DS.Y)/2
ymax = np.max(DS.Y)/2

# spacing of meshgrid (in degrees)
dx = np.minimum(X[0, 1]-X[0, 0], Y[1, 0]-Y[0, 0])/30
dy = dx

x_grid = np.arange(xmin, xmax+dx, dx)
y_grid = np.arange(ymin, ymax+dy, dy)

# Specify ratio of auxiliary grid
ratio_auxiliary_grid = 0.1

params_ds = time, x_grid, y_grid, ratio_auxiliary_grid
DS._init_ds(params_ds)

Wall time: 970 µs


In [ ]:
FTLE_min, FTLE_max = DS._FTLE_()

=================Trajectories over meshgrid=================


  0%|          | 0/286 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  3.0min


In [ ]:
fig = plt.figure(figsize = (14, 8))
ax = plt.axes()
cax = ax.contourf(DS.trajectory_grid[:, :, 0, 0], DS.trajectory_grid[:, :, 1, 0], FTLE_max, levels = 600, cmap = "Blues")
#cax = ax.scatter(DS.trajectory_grid[:, :, 0, 0].ravel(), DS.trajectory_grid[:, :, 1, 0].ravel(), c = FTLE_max.ravel())
plt.colorbar(cax, ticks = np.linspace(0, 0.05, 6))
plt.show()

In [ ]:
PRA = DS._PRA_()

In [ ]:
fig = plt.figure(figsize = (14, 8))
ax = plt.axes()
cax = ax.contourf(DS.trajectory_grid[:, :, 0, 0], DS.trajectory_grid[:, :, 1, 0], PRA, levels = 600, cmap = "gist_rainbow")
#cax = ax.scatter(DS.trajectory_grid[:, :, 0, 0].ravel(), DS.trajectory_grid[:, :, 1, 0].ravel(), c = FTLE_max.ravel())
plt.colorbar(cax, ticks = np.linspace(0, np.pi, 8))
plt.show()

In [ ]:
TRA_bar = DS._TRA_bar_()

In [ ]:
fig = plt.figure(figsize = (14, 8))
ax = plt.axes()
cax = ax.contourf(DS.trajectory_grid[:, :, 0, 0], DS.trajectory_grid[:, :, 1, 0], TRA_bar, levels = 600, cmap = "gist_rainbow")
#cax = ax.scatter(DS.trajectory_grid[:, :, 0, 0].ravel(), DS.trajectory_grid[:, :, 1, 0].ravel(), c = FTLE_max.ravel())
plt.colorbar(cax)
plt.show()

In [ ]:
TSE = DS._TSE_()

In [ ]:
TSE_bar = DS._TSE_bar_()

In [ ]:
fig = plt.figure(figsize = (14, 8))
ax = plt.axes()
cax = ax.contourf(DS.trajectory_grid[:, :, 0, 0], DS.trajectory_grid[:, :, 1, 0], TRA_bar, levels = 600, cmap = "gist_rainbow")
plt.colorbar(cax)
plt.show()

In [ ]:
extrema_x, extrema_y = DS._ridge_trench(FTLE_max, threshold = 0.02, type = "ridge", method = "gradient", 
                                        resolution = 2, ds = .5, n_iterations = 100)
plt.scatter(extrema_x, extrema_y, s = .5)
plt.show()